In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive")
print(os.getcwd)

<built-in function getcwd>


In [ ]:

import pandas as pd #manipulation et l'analyse de données tabulaires
import re # manipulation de chaînes de caractères
import spacy #  traitement du langage naturel (NLP)
from textblob import TextBlob # traitement du langage naturel (NLP) qui offre une interface simple
import emoji # travailler avec des emojis
from textblob import Blobber #  étend les fonctionnalités de TextBlob pour la langue française
from textblob_fr import PatternTagger, PatternAnalyzer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder # convertir des valeurs de texte en valeurs numériques



df = pd.read_csv("C:/Users/W10/Downloads/sae n/commentaires_films_series.csv")

# Charger le modèle de langue française
nlp = spacy.load('fr_core_news_sm')
# modèle pré-entraîné qui comprend des fonctionnalités telles que la
# tokenisation, l'étiquetage grammatical, la reconnaissance d'entités
# nommées, la lemmatisation, etc

# Fonction pour compter les émoticônes
def compter_emoticones(texte):
    return emoji.emoji_count(texte)
# compter_emoticones qui prend une chaîne de caractères (texte)
# en entrée et compte le nombre d'émoticônes présentes dans cette
# chaîne


# Fonction pour compter les mots
def compter_mots(texte):
    return len(texte.split())
# compter_mots qui prend une chaîne de caractères (texte) en entrée et
# compte le nombre de mots présents dans cette chaîne

# Fonction pour compter les verbes, entités nommées et adjectifs
def compter_pos_entites(texte):
    doc = nlp(texte)
    nb_verbes = len([token for token in doc if token.pos_ == "VERB"])
    nb_entites = len(doc.ents)
    nb_adjectifs = len([token for token in doc if token.pos_ == "ADJ"])
    return nb_verbes, nb_entites, nb_adjectifs
# cette ligne retourne un tuple contenant le nombre de verbes,
# d'entités nommées et d'adjectifs trouvés dans le texte


import spacy
from textblob import TextBlob

# Chargez le modèle de langue français de spaCy
nlp = spacy.load('fr_core_news_sm')

def temps_dominant(texte):
    doc = nlp(texte)
    temps_verbes = {'present': 0, 'past': 0, 'future': 0}

    for token in doc:
        if token.pos_ == 'VERB' or token.pos_ == 'AUX':
            if token.morph.get("Tense") == ['Pres']:
                temps_verbes['present'] += 1
            elif token.morph.get("Tense") == ['Past']:
                temps_verbes['past'] += 1
            elif token.morph.get("Mood") == ['Cnd'] or token.text.lower() in ['sera', 'auront', 'iront']:
                temps_verbes['future'] += 1

    # Trouvez le temps le plus utilisé et renvoyez-le
    return max(temps_verbes, key=temps_verbes.get), temps_verbes

# lorsqu'on appelle cette fonction avec une chaîne de caractères en
# entrée, elle retourne le temps verbal dominant dans le texte ainsi
# que le décompte de chaque temps verbal


from textblob import TextBlob
# utilisée pour l'analyse de sentiment

def analyser_sentiment_verbes(texte):
    doc = nlp(texte)
    sentiments = {'positifs': 0, 'negatifs': 0}

    for token in doc:
        if token.pos_ == 'VERB':
            verdict = TextBlob(str(token))
            if verdict.sentiment.polarity > 0:
                sentiments['positifs'] += 1
            elif verdict.sentiment.polarity < 0:
                sentiments['negatifs'] += 1

    return sentiments
# retourne le dictionnaire sentiments contenant le nombre de verbes
# positifs et négatifs trouvés dans le texte

# cette fonction avec une chaîne de caractères en entrée, elle
#retourne un dictionnaire contenant le nombre de verbes positifs et
# négatifs dans le texte.

# Assurez-vous que 'nlp' est bien défini et initialisé, par exemple:
# nlp = spacy.load('fr_core_news_sm') ou le modèle de votre choix



# Application des fonctions
df['Nb_Mots'] = df['Commentaire'].apply(compter_mots)
df['Nb_Emoticones'] = df['Commentaire'].apply(compter_emoticones)

# Compter les verbes, entités nommées et adjectifs et les ajouter au DataFrame
df[['Nb_Verbes', 'Nb_Entites_Nommees', 'Nb_Adjectifs']] = df['Commentaire'].apply(
    lambda x: pd.Series(compter_pos_entites(x))
)

# Appliquer les fonctions sur le DataFrame
# Appliquer la fonction modifiée pour obtenir le temps dominant et le décompte des verbes
df['Temps_Dominant'], df['Compte_Temps'] = zip(*df['Commentaire'].apply(temps_dominant))

# Créer des colonnes individuelles pour chaque temps à partir du dictionnaire retourné
df['Nb_Verbes_Present'] = df['Compte_Temps'].apply(lambda x: x['present'])
df['Nb_Verbes_Past'] = df['Compte_Temps'].apply(lambda x: x['past'])
df['Nb_Verbes_Future'] = df['Compte_Temps'].apply(lambda x: x['future'])

# Vous pouvez ensuite supprimer la colonne 'Compte_Temps' si elle n'est pas nécessaire
df.drop(columns=['Compte_Temps'], inplace=True)

# Appliquez la fonction à votre DataFrame
df['Sentiment_Verbes'] = df['Commentaire'].apply(analyser_sentiment_verbes)


# Afficher le DataFrame pour vérifier
df


# Enregistrer le DataFrame en fichier CSV
df.to_csv('C:/Users/gaous/Desktop/BUT 3/S6/R6 06 machine learning texte/commentaire_avec_critères.csv', index=False)

df.columns

#
# Créer la colonne cible en fonction du score
df['Cible'] = (df['Note'] > 3.5).astype(int)
# Initialiser le LabelEncoder
le = LabelEncoder()

# Supposons que 'Colonne_Catégorielle' est le nom de votre colonne catégorielle
df['Temps_Dominant'] = le.fit_transform(df['Temps_Dominant'])

# Sélectionner les caractéristiques pour la régression logistique

X = df[['Nb_Mots', 'Nb_Emoticones', 'Nb_Verbes','Nb_Entites_Nommees', 'Nb_Adjectifs',
        'Temps_Dominant','Nb_Emoticones', 'Nb_Verbes_Present', 'Nb_Verbes_Past', 'Nb_Verbes_Future']]
y = df['Cible']

# Supposons que 'X' sont les caractéristiques et 'y' est la variable cible.
# Diviser les données, en utilisant à la fois 'train_size' et 'test_size', avec stratification et mélange
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.17,     # 20% des données comme ensemble de test
    train_size=0.83,    # 80% des données comme ensemble d'entraînement
    random_state=42,   # Pour la reproductibilité
    shuffle=True,      # Mélanger les données avant de diviser
    stratify=y         # Pour conserver la proportion des classes dans les splits
)

# Créer et entraîner le modèle de régression logistique
model = LogisticRegression()
model.fit(X_train, y_train)

# Prédire les valeurs pour l'ensemble de test
y_pred = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"L'exactitude du modèle est : {accuracy}")
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, log_loss

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Matrice de confusion :\n{conf_matrix}")

# Rapport de classification
class_report = classification_report(y_test, y_pred)
print(f"Rapport de classification :\n{class_report}")

# Précision, Rappel, Score F1
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Précision : {precision}")
print(f"Rappel : {recall}")
print(f"Score F1 : {f1}")

# ROC AUC
y_pred_proba = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC : {roc_auc}")

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Prédictions de probabilité pour la classe positive
y_pred_proba = model.predict_proba(X_test)[:,1]

# Calcul des taux de vrais positifs et faux positifs
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Calcul de l'aire sous la courbe (AUC)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


# Log Loss
logloss = log_loss(y_test, y_pred_proba)
print(f"Log Loss : {logloss}")


Deuxième partie de la SAE

Charger les données

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/My Drive/SAE 6 EMS 1 Modélisation statistique IA sur texte/sae/commentaire_avec_critères.csv")

Vérification du chargement

In [ ]:
df

,URL,Note,Commentaire,Nb_Mots,Nb_Emoticones,Nb_Verbes,Nb_Entites_Nommees,Nb_Adjectifs,Temps_Dominant,Nb_Verbes_Present,Nb_Verbes_Past,Nb_Verbes_Future
0,https://www.allocine.fr/film/fichefilm-267218/...,0.5,"Alors comment dire...En fait non, il n'y a pas...",64,0,8,2,7,present,6,3,0
1,https://www.allocine.fr/film/fichefilm-267218/...,1.5,"L'équipe de ""Sound of Freedom"" a fini par adme...",114,0,14,1,11,present,8,4,0
2,https://www.allocine.fr/film/fichefilm-267218/...,1.0,Je ne sais pas pkoi je dois rédiger une critiq...,43,0,7,0,7,present,4,2,0
3,https://www.allocine.fr/film/fichefilm-267218/...,1.0,Tout n'est que cliché dans ce film : un scénar...,78,0,8,1,9,present,6,3,0
4,https://www.allocine.fr/film/fichefilm-267218/...,0.5,Vraiment pas fou pour un premier rendez vous a...,23,1,1,1,4,present,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8821,https://www.allocine.fr/series/ficheserie-290/...,5.0,"Même pas besoin de critique, les Simpson parle...",10,0,1,1,0,present,1,0,0
8822,https://www.allocine.fr/series/ficheserie-290/...,5.0,merci tout simplement ! rien à dire qui n'est ...,23,0,2,1,1,present,2,1,0
8823,https://www.allocine.fr/series/ficheserie-290/...,5.0,Une série toujours aussi marante qui en 25 ans...,15,0,1,0,1,present,1,1,0
8824,https://www.allocine.fr/series/ficheserie-290/...,4.0,"Contrairement aux apparences, série bourrée de...",17,0,2,0,4,present,1,0,0


Chargement des packages

In [ ]:
import numpy as np
import collections
import os, sys
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras.saving import load_model

Chargement du modèle

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Chemin vers le fichier du modèle pré-entraîné
chemin_vers_modele = "/content/drive/My Drive/SAE 6 EMS 1 Modélisation statistique IA sur texte/sae/tf_model.h5"

In [ ]:
# Charger le modèle pré-entraîné
modele_charge = load_model("/content/drive/My Drive/SAE 6 EMS 1 Modélisation statistique IA sur texte/sae/tf_model.h5")

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x7cf7250ecbe0>.

In [ ]:
# Afficher les détails du modèle
modele_charge.summary()

In [ ]:


from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Activation
from tensorflow.keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam

learning_rate= 1e-3
inputs = Input(shape=(None,))
embedding = Embedding(input_dim=english_vocab_size+1,output_dim=32,
input_length=tokenized_english_sentences.shape[1])(inputs)
encoder = Bidirectional(GRU(256, return_sequences=False))(embedding)
context = RepeatVector(max_len_sentence_french)(encoder)
decoder = Bidirectional(GRU(256, return_sequences=True))(context)
dense = TimeDistributed(Dense(french_vocab_size+1))(decoder)
outputs = Activation("softmax")(dense)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate),
metrics=['accuracy'])
model.summary()

Faire tourner le programme

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

history = model.fit(tokenized_english_sentences,
tokenized_french_sentences, batch_size=1024, epochs=40,
validation_split=0.2,
callbacks=[EarlyStopping(monitor='val_loss', patience=3)])
model.save("/content/drive/My Drive/ML S6 BUT 3/my_model.keras_2")